In [ ]:
# 셀 2) import 및 YouTube API 클라이언트 초기화
import pandas as pd
import os
import time
from googleapiclient.discovery import build
from tqdm.notebook import tqdm

# API 키 설정 (환경변수 없으면 직접 넣어도 됨)
api_key = 'API_KEY'
youtube = build('youtube', 'v3', developerKey=api_key)

In [2]:
# 셀 3) 데이터 로드 및 video_id 추출 함수 정의
df = pd.read_csv('AAA_release.csv')

In [3]:
# 셀 4) 게시일을 가져오는 함수 (50개씩 배치 + tqdm 진행바)
def fetch_publish_dates(video_ids):
    publish_dates = {}
    chunks = [video_ids[i:i+50] for i in range(0, len(video_ids), 50)]
    for chunk in tqdm(chunks, desc='Fetching chunks'):
        res = youtube.videos().list(
            part='snippet',
            id=','.join(chunk)
        ).execute()
        for item in res.get('items', []):
            publish_dates[item['id']] = item['snippet']['publishedAt']
        time.sleep(1)  # 할당량 보호
    return publish_dates

video_ids = df['video_link'].unique().tolist()
publish_dates = fetch_publish_dates(video_ids)

Fetching chunks:   0%|          | 0/72 [00:00<?, ?it/s]

In [4]:
# 셀 5) DataFrame에 게시일 매핑 후 저장
df['youtube_uploaded_date'] = df['video_link'].map(publish_dates)

In [5]:
df.head()

publisher                 game_names      video_type   video_link  \
0    Capcom              Lost Planet 3         Trailer  igfmUe-IBeQ   
1    Capcom              Lost Planet 3  Gameplay video  8anIGeV18HE   
2    Capcom              Lost Planet 3  Gameplay video  afZAmzYWEu4   
3    Capcom  Call of Duty: Finest Hour  Gameplay Video  2f8S7gPK4b4   
4    Capcom  Call of Duty: Finest Hour         Trailer  qDCADmoCec8   

                                     video_url game_release_date  \
0  https://www.youtube.com/watch?v=igfmUe-IBeQ        2013-08-26   
1  https://www.youtube.com/watch?v=8anIGeV18HE        2013-08-26   
2  https://www.youtube.com/watch?v=afZAmzYWEu4        2013-08-26   
3  https://www.youtube.com/watch?v=2f8S7gPK4b4        2004-11-16   
4  https://www.youtube.com/watch?v=qDCADmoCec8        2004-11-16   

  youtube_uploaded_date  
0  2012-08-14T11:14:55Z  
1  2016-02-28T19:27:03Z  
2  2016-02-29T13:49:12Z  
3  2019-12-16T17:30:00Z  
4  2011-05-19T22:57:07Z

In [6]:
df.to_csv('AAA_upload.csv', index=False)
print("✅ 완료! 'AAA_upload.csv'에 저장됨")

✅ 완료! 'sample_with_publish_dates.csv'에 저장됨
